# 교재 Part 6

In [65]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import folium
from config.settings import DATA_DIR    # 그대로 사용해도 됨

In [250]:
# 데이터 파일을 불러오기
df = pd.read_excel('data/city_pop.xlsx')    # 시도별 전출입 인구수
elec = pd.read_excel('data/elec_energy.xlsx')    # 남북한 전력 이용
auto = pd.read_csv('data/auto-mpg.csv', header=None)     # 자동차 정보
auto.columns = ['mpg', 'cylinders', 'displacement', 'hp', 'weight', 'acceleration', 'model_year', 'origin', 'name']
ttn = sns.load_dataset('titanic')
col = pd.read_excel('data/col_location.xlsx', index_col=0)     # 서울 대학교 위치정보
ggd = pd.read_excel('data/gg_pop.xlsx')        # 경기도 인구 정보
stock = pd.read_csv('data/stock.csv')
stock_d = pd.read_excel('data/stock-data.xlsx')
stock_p = pd.read_excel(DATA_DIR[0] + '/stock-price.xlsx')
stock_v = pd.read_excel(DATA_DIR[0] + '/stock-valuation.xlsx')

In [214]:
class Util:
    def add_10(n):
        return n + 10

    def add_both(a, b):
        return a + b
    
    def missing_values(series):
        return series.isnull()
    
    def min_max(series):
        return series.max() - series.min()
    
    def kpl_cyl(mpg, cyl):
        return mpg * 1.6/3.7 + cyl
    
    def info(df):
        return df.info()           #여기 함수????????????????????????????????????????????????????
    
    def zscore(x):
        return (x - x.mean()) / x.std()

## Apply - Series에 elementwise / Dataframe에 serieswise로 함수 적용

In [246]:
class P6:
    def p218(self):
        df = ttn.loc[:, ['age', 'fare']]
        df['ten'] = 10
        print(df)

        # Apply 함수 - Series에 적용
        sr1 = df['age'].apply(Util.add_10)
        #print(sr1)
        sr2 = df['age'].apply(Util.add_both, b=10)
        #print(sr2)
        sr3 = df['age'].apply(lambda x: Util.add_10(x))         # x = df['age']
        #print(sr3)          # sr1, 2, 3 모두 같은 결과
  

        # Dataframe에도 적용 가능
        x = ttn.loc[:, ['age', 'fare']]
        df0 = x.apply(Util.add_10, axis=0)
        #print(df0)
        df1 = x.apply(Util.add_10, axis=1)
        #print(df1)   # df0과 같은 결과

        miss = x.apply(Util.missing_values, axis=1)    # axis=0, 1 모두 결과 같다
        #print(miss)
        
        
        # 어느 경우에 axis=0, 1이 차이를 보일까??
        diff0 = x.apply(Util.min_max, axis=0)
        #print(diff0)       # 각 열의 최대 - 최소
        
        diff1 = x.apply(Util.min_max, axis=1)
        #print(diff1)       # 각 행의 최대 - 최소
        
        
        add10 = df.apply(lambda x: Util.add_both(x['age'], x['ten']), axis=1)   # x=df의 각 행
        #print(add10)
        
if __name__ == '__main__':
    P6().p218()

      age     fare  ten
0    22.0   7.2500   10
1    38.0  71.2833   10
2    26.0   7.9250   10
3    35.0  53.1000   10
4    35.0   8.0500   10
..    ...      ...  ...
886  27.0  13.0000   10
887  19.0  30.0000   10
888   NaN  23.4500   10
889  26.0  30.0000   10
890  32.0   7.7500   10

[891 rows x 3 columns]
0      32.0
1      48.0
2      36.0
3      45.0
4      45.0
       ... 
886    37.0
887    29.0
888     NaN
889    36.0
890    42.0
Length: 891, dtype: float64


In [251]:
class P6:
    def test(self):
        display(auto.head()) 
        
        # mpg컬럼에 kpl_cyl함수를 적용해서 kpl 컬럼을 추가하시오.
        #auto['kpl'] = auto['mpg'].apply(Util.kpl_cyl, cyl=auto['cylinders'])  # Series에 apply는 elementwise이기 때문에 cyl이 series는 안된다
        auto['kpl'] = auto[['mpg']].apply(Util.kpl_cyl, cyl=auto['cylinders']) # Dataframe에 적용해서 serieswise로 적용되도록 함
        #display(auto.head())
       
        # 강사님 방법
        auto['kpl_t'] = auto.apply(lambda x: Util.kpl_cyl(x['mpg'], x['cylinders']) , axis=1)
        display(auto.head())
        
if __name__ == '__main__':
    P6().test()

,mpg,cylinders,displacement,hp,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


,mpg,cylinders,displacement,hp,weight,acceleration,model_year,origin,name,kpl,kpl_t
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,15.783784,15.783784
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,14.486486,14.486486
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,15.783784,15.783784
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,14.918919,14.918919
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,15.351351,15.351351


## Applymap - Dataframe에 elementwise로 함수 적용

In [43]:
class P6:
    def p221(self):
        df = ttn.loc[:, ['age', 'fare']]
        
        # Applymap 함수 - dataframe에 elementwise로 적용
        df1 = df.applymap(Util.add_10)
        #print(df1)
        
        df2 = df.applymap(Util.min_max)   # 불가능
        print(df2)
    
if __name__ == '__main__':
    P6().p221()

AttributeError: 'float' object has no attribute 'max'

## Pipe - Series/Dataframe 자체에 함수 적용

In [135]:
class P6:
    def p226(self):
        # Series에 적용
        #print(ttn['age'].pipe(max))
        
        # Dataframe에 적용
        #ttn.pipe(Util.info)
        
        #ttn.loc[:, ['age', 'fare']].pipe(Util.total)       # total 함수가 머야..............
        
        
    
if __name__ == '__main__':
    P6().p226()

80.0


## 컬럼 순서 정렬

In [62]:
class P6:
    def p229(self):
        # 컬럼 정렬
        df = ttn.loc[:, 'survived':'age']
        print(df)
        print(df.columns.values)
        sorted_col = sorted(df.columns)
        print(sorted_col)       # 알파벳 순으로 정렬됨
        sorted_df = df[sorted_col]     # 정렬된 순으로 컬럼을 불러온다
        print(sorted_df)
        
     
if __name__ == '__main__':
    P6().p229()

     survived  pclass     sex   age
0           0       3    male  22.0
1           1       1  female  38.0
2           1       3  female  26.0
3           1       1  female  35.0
4           0       3    male  35.0
..        ...     ...     ...   ...
886         0       2    male  27.0
887         1       1  female  19.0
888         0       3  female   NaN
889         1       1    male  26.0
890         0       3    male  32.0

[891 rows x 4 columns]
['survived' 'pclass' 'sex' 'age']
['age', 'pclass', 'sex', 'survived']
      age  pclass     sex  survived
0    22.0       3    male         0
1    38.0       1  female         1
2    26.0       3  female         1
3    35.0       1  female         1
4    35.0       3    male         0
..    ...     ...     ...       ...
886  27.0       2    male         0
887  19.0       1  female         1
888   NaN       3  female         0
889  26.0       1    male         1
890  32.0       3    male         0

[891 rows x 4 columns]


## 열 분리

In [91]:
class P6:
    def p232(self):
        print(stock_d.head())
        #print(stock_d['연월일'].dtypes)   # datetime64
        stock_d['연월일'] = stock_d['연월일'].astype('str')
        #print(stock_d.head())
        date = stock_d['연월일'].str.split('-')
        #print(type(date))    # series
        print(date.head())
        
        # 연월일 분리
        stock_d['연'] = date.str.get(0)
        stock_d['월'] = date.str.get(1)
        stock_d['일'] = date.str.get(2)
        display(stock_d.head())
        
        
if __name__ == '__main__':
    P6().p232()

          연월일   당일종가  전일종가     시가     고가     저가     거래량     연   월   일
0  2018-07-02  10100   600  10850  10900  10000  137977  2018  07  02
1  2018-06-29  10700   300  10550  10900   9990  170253  2018  06  29
2  2018-06-28  10400   500  10900  10950  10150  155769  2018  06  28
3  2018-06-27  10900   100  10800  11050  10500  133548  2018  06  27
4  2018-06-26  10800   350  10900  11000  10700   63039  2018  06  26
0    [2018, 07, 02]
1    [2018, 06, 29]
2    [2018, 06, 28]
3    [2018, 06, 27]
4    [2018, 06, 26]
Name: 연월일, dtype: object


,연월일,당일종가,전일종가,시가,고가,저가,거래량,연,월,일
0,2018-07-02,10100,600,10850,10900,10000,137977,2018,07,02
1,2018-06-29,10700,300,10550,10900,9990,170253,2018,06,29
2,2018-06-28,10400,500,10900,10950,10150,155769,2018,06,28
3,2018-06-27,10900,100,10800,11050,10500,133548,2018,06,27
4,2018-06-26,10800,350,10900,11000,10700,63039,2018,06,26


## Filtering

In [101]:
class P6:           # 다시확인???????????????????????????????
    def p234(self):
        m1 = (ttn['age'] >= 10) & (ttn['age'] < 20)         # 10대만 추림
        #display(ttn[m1])
        m2 = (ttn['age'] >= 10) & (ttn['sex'] == 'female')
        #display(ttn[m2])
        m3 = (ttn['age'] < 10) | (ttn['age'] >= 60)
        #display(ttn[m3]) 
        #display(ttn.loc[m3, ['age', 'sex', 'alone']])
        
        m4 = ttn['pclass'] == 1
        #display(ttn.loc[m4, ['age', 'sex']])
        
        # 출력 설정 변경 - 최대 열의 개수 지정해서 모든 열 볼 수 있도록함
        pd.set_option('display.max_columns', 20)
        
        # isin 함수
        tf = ttn['embark_town'].isin(['Southampton', 'Queenstown'])
        result = ttn[tf]
        display(result.tail())
        
        
if __name__ == '__main__':
    P6().p234()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
885,0,3,female,39.0,0,5,29.125,Q,Third,woman,False,NaN,Queenstown,no,False
886,0,2,male,27.0,0,0,13.000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.450,S,Third,woman,False,NaN,Southampton,no,False
890,0,3,male,32.0,0,0,7.750,Q,Third,man,True,NaN,Queenstown,no,True


## Dataframe 합치기 - Concat, Join, Merge

In [106]:
df1 = pd.DataFrame({'a': ['a0', 'a1', 'a2', 'a3'],
                    'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3']
                    }, index=[0, 1, 2, 3])

df2 = pd.DataFrame({'a': ['a2', 'a3', 'a4', 'a5'],
                    'b': ['b2', 'b3', 'b4', 'b5'],
                    'c': ['c2', 'c3', 'c4', 'c5'],
                    'd': ['d2', 'd3', 'd4', 'd5']
                    }, index=[2, 3, 4, 5])
    
display(df1)
display(df2)

,a,b,c
0,a0,b0,c0
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


,a,b,c,d
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5


In [158]:
sr1 = pd.Series(['e0', 'e1', 'e2', 'e3'], name='e')    # index= [0, 1, 2, 3]
sr2 = pd.Series(['f0', 'f1', 'f2'], name='f', index=[3, 4, 5])
sr3 = pd.Series(['go', 'g1', 'g2', 'g3'], name='g')

In [160]:
class P6:
    def p239(self):
        # Concat 함수 - Dataframe
        r1 = pd.concat([df1, df2], axis=1, ignore_index=True, join='inner')   # igonore_index=True이면 무조건 숫자 인덱스로 바뀜
        #display(r1)
    
        # Concat - Series
        r2 = pd.concat([df1, sr1], axis=1)
        #display(r2)
        r3 = pd.concat([df1, sr2], axis=1, join='inner')
        #display(r3)
        r4 = pd.concat([df2, sr2], axis=1)
        #display(r4)
        r5 = pd.concat([df2, sr3], axis=1)
        display(r5)
        
if __name__ == '__main__':
    P6().p239()

,a,b,c,d,g
0,NaN,NaN,NaN,NaN,go
1,NaN,NaN,NaN,NaN,g1
2,a2,b2,c2,d2,g2
3,a3,b3,c3,d3,g3
4,a4,b4,c4,d4,NaN
5,a5,b5,c5,d5,NaN


In [192]:
class P6:
    def p245(self):
        # Merge 함수
        display(stock_p.head(), stock_v.head())
        df1 = pd.merge(stock_p, stock_v, how='right')
        #display(df1)
        df2 = pd.merge(stock_p, stock_v, left_on='stock_name', right_on='name', how='right')
        #display(df2)
    
        # 5만원 미만인 종목에 대해서만 두 데이터프레임을 합친다
        p5 = stock_p[stock_p['price'] < 50000]
        #display(p5)
        df3 = pd.merge(p5, stock_v, on='id', how='outer')
        #display(df3)
        
        
if __name__ == '__main__':
    P6().p245()

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000
1,130960,CJ E&M,58540.666667,98900
2,138250,엔에스쇼핑,14558.666667,13200
3,139480,이마트,239230.833333,254500
4,142280,녹십자엠에스,468.833333,10200


,id,name,eps,bps,per,pbr
0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,136480,하림,274.166667,3551,11.489362,0.887074
2,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,18268.166667,295780,13.931338,0.860437
4,145990,삼양사,5741.000000,108090,14.283226,0.758627


In [193]:
class P6:
    def p252(self):
        # Join 함수
        # on: df의 "컬럼"이나 index level를 지정하면, other의 index에 겹쳐진다. 디폴트는 index-on-index
        st_p = stock_p.set_index('id')
        st_v = stock_v.set_index('id')
        display(st_p, st_v)
        df1 = st_p.join(st_v, how='inner')
        display(df1.head())
    
if __name__ == '__main__':
    P6().p252()

,stock_name,value,price
id,,,
128940,한미약품,59385.666667,421000
130960,CJ E&M,58540.666667,98900
138250,엔에스쇼핑,14558.666667,13200
139480,이마트,239230.833333,254500
142280,녹십자엠에스,468.833333,10200
145990,삼양사,82750.000000,82000
185750,종근당,40293.666667,100500
192400,쿠쿠홀딩스,179204.666667,177500
199800,툴젠,-2514.333333,115400


,name,eps,bps,per,pbr
id,,,,,
130960,CJ E&M,6301.333333,54068,15.695091,1.829178
136480,하림,274.166667,3551,11.489362,0.887074
138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
139480,이마트,18268.166667,295780,13.931338,0.860437
145990,삼양사,5741.000000,108090,14.283226,0.758627
161390,한국타이어,5648.500000,51341,7.453306,0.820007
181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
185750,종근당,3990.333333,40684,25.185866,2.470259
204210,모두투어리츠,85.166667,5335,40.802348,0.651359


,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


## Group 연산

In [275]:
class P6:
    def p254(self):
        df = ttn[['age', 'sex', 'class', 'fare', 'survived']]
        display(df.head())
        
        # Group 함수
        group = df.groupby(['class'])
        print(group)           # DataFrameGroupby object
        #for data in group:    # 만들어진 그룹을 반복 - (group 이름, group의 subframe) tuple이 group object
        #    display(data)
        #    print(type(data))
            
        r1 = df.groupby(['class']).mean()
        #print(r1)
        
        # 원하는 class의 내용만 확인하기
        sec1 = group.get_group('Second')
        #print(sec1)
        
        #sec2 = group.loc['Second']       # 이거 뭐였냐고??????????????///
        #print(sec2)
        
        r2 = df.groupby(['class', 'sex']).mean()
        #print(r2)
        #print(r2.loc['First'])               # indexing
        #print(r2.loc[('First', 'female')])   # multi-indexing
        
        th_female = df.groupby(['class', 'sex']).get_group(('Third', 'female'))
        #print(th_female)
        
        
if __name__ == '__main__':
    P6().p254()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [271]:
class P6:
    def p261(self):
        df = ttn[['age', 'sex', 'class', 'fare', 'survived']]
        s = df.groupby(['class']).std() 
        #print(s)
    
        s2 = df.groupby(['class'])['fare']            # SeriesGroupby object
        #for s in s2:
        #    print(s)
        
        s2 = df.groupby(['class'])['fare'].std() 
        #print(s2)
        s3 = df.groupby(['class']).fare.std()
        #print(s3)    # s2와 같은 결과
        
        
        # Group 전체에 함수 적용 - agg 메소드
        r1 = df.groupby(['class']).agg(Util.min_max)
        #print(r1)
        
        r2 = df.groupby(['class']).agg(['min', 'max'])
        #print(r2)
        
        r3 = df.groupby(['class']).agg({'fare':['min', 'max'], 'age':'mean'})
        #print(r3)
        
        
        # transform 메소드??????????????????????????????????????????????????    
        r4 = df.groupby(['class']).transform(Util.zscore)   # class별 데이터프레임 중, object 타입인 sex, class를 제외하고 연산
        #print(r4)
        
        r5 = df.groupby(['class'])['age'].transform(Util.zscore)
        #print(r5)
        
        
        # filter메소드
        r6 = df.groupby(['class']).filter(lambda x: len(x) >= 200)   # x: class별 데이터프레임
        #print(r6)
        
        
if __name__ == '__main__':
    P6().p261()

('First', 1      71.2833
3      53.1000
6      51.8625
11     26.5500
23     35.5000
        ...   
871    52.5542
872     5.0000
879    83.1583
887    30.0000
889    30.0000
Name: fare, Length: 216, dtype: float64)
('Second', 9      30.0708
15     16.0000
17     13.0000
20     26.0000
21     13.0000
        ...   
866    13.8583
874    24.0000
880    26.0000
883    10.5000
886    13.0000
Name: fare, Length: 184, dtype: float64)
('Third', 0       7.2500
2       7.9250
4       8.0500
5       8.4583
7      21.0750
        ...   
882    10.5167
884     7.0500
885    29.1250
888    23.4500
890     7.7500
Name: fare, Length: 491, dtype: float64)
